In [73]:
def load_items(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    items = {}
    for line in lines:
        line = line.strip()
        item_id, attribute_1, attribute_2 = line.split('|')
        item_id = int(item_id)
        if attribute_1 == 'None':
            attribute_1 = None
        if attribute_2 == 'None':
            attribute_2 = None
        items[item_id] = (attribute_1, attribute_2)

    return items

items = load_items('./data/itemAttribute.txt')

In [1]:
from tqdm import tqdm
import numpy as np  

In [2]:
# read rating data
# construct data{} and item_rating{}
data_path = './data/train.txt'

centerlize = False  # centerlize the rating data or not

train_lines = 0
all_avg = 0
item_rating= {}

with open(data_path, 'r') as f:
    lines = f.readlines()
    user_id = None
    train_lines = len(lines)
    data = {}
    for line in lines:
        line = line.strip()
        if '|' in line:  # user line
            train_lines -= 1
            if(user_id != None and centerlize == True):
                avg = data[user_id]['sum'] / data[user_id]['num_ratings']
                data[user_id]['ratings'].update({k: v - avg for k, v in data[user_id]['ratings'].items()})
                data[user_id]['norm'] = sum(x**2 for x in data[user_id]['ratings'].values())**0.5

            user_id, num_ratings = line.split('|')
            user_id = int(user_id)
            data[user_id] = {}
            data[user_id]['num_ratings'] = int(num_ratings)
            data[user_id]['ratings'] = {}
            data[user_id]['sum'] = 0
        else:  # rating line
            item_id, score = map(int, line.split())
            data[user_id]['ratings'][item_id] = score
            data[user_id]['sum'] += score
            all_avg += score
            if item_id not in item_rating:
                item_rating[item_id] = {'num': 0, 'sum': 0}
            item_rating[item_id]['num'] += 1
            item_rating[item_id]['sum'] += score
    if centerlize == True:
        avg = data[user_id]['sum'] / data[user_id]['num_ratings']
        data[user_id]['ratings'].update({k: v - avg for k, v in data[user_id]['ratings'].items()})
        data[user_id]['norm'] = sum(x**2 for x in data[user_id]['ratings'].values())**0.5

all_avg /= train_lines
lines = None

In [3]:
bias = {}
for userid in data.keys():
    bias[userid] = data[userid]['sum']/data[userid]['num_ratings'] - all_avg
item_bias = {}
for itemid in item_rating.keys():
    item_bias[itemid] = item_rating[itemid]['sum']/item_rating[itemid]['num'] - all_avg

In [ ]:
#cacluate similarity between users
#takes 1 hour 20mins
similarity = {}
for i, (userid1, user1_data) in tqdm(enumerate(data.items()), total=len(data)):
    similarity[userid1] = {}
    for j, (userid2, user2_data) in enumerate(data.items()):
        if  user1_data['norm'] == 0 or  user2_data['norm'] == 0:
            similarity[userid1][userid2] = 0
            continue
        if i >= j:
            continue
        else:
            cos_sim = 0.0
            for item, rating in user1_data['ratings'].items():
                if item in user2_data['ratings']:
                    cos_sim += rating * user2_data['ratings'][item]
            cos_sim = cos_sim / (user1_data['norm'] * user2_data['norm'])
            similarity[userid1][userid2] = cos_sim

100%|██████████| 19835/19835 [1:23:37<00:00,  3.95it/s]  


In [ ]:
# save similarity matrix
np.save('similarity.npy', similarity)

In [4]:
# load similarity matrix
similarity = np.load('similarity.npy', allow_pickle=True).item()

In [5]:
max(similarity[0].values())

0.27493

In [5]:
#read test data
test_input = {}
with open('./data/test.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()  # 去除行尾的换行符
        if '|' in line:
            # 这是一个用户的开始
            userid, num_items = line.split('|')  # 分割用户ID和评分项目数量
            test_input[userid] = {'num_items': int(num_items), 'items': []}
        else:
            # 这是一个项目ID
            itemid = line
            test_input[userid]['items'].append(itemid)
lines = None

In [6]:
import heapq

def find_top_n_keys(dictionary, n):
    top_n_items = heapq.nlargest(n, ((k, v) for k, v in dictionary.items()), key=lambda item: item[1])
    top_n_keys = [key for (key, value) in top_n_items ]
    return top_n_keys

In [ ]:
test_output = {}

for user_id, user_data in (tqdm(test_input.items())):
    user_id = int(user_id)
    test_output[user_id] = {}
    N_neighbor = find_top_n_keys(similarity[user_id], 500)
    for item_id in user_data['items']:
        item_id = int(item_id)
        eval = 0
        sim_sum = 0
        for neighbor in N_neighbor:
            if item_id in data[neighbor]['ratings']:
                sim_sum += similarity[user_id][neighbor]
                baseline = data[neighbor]['sum']/data[neighbor]['num_ratings'] +  item_rating[item_id]['sum']/item_rating[item_id]['num'] - all_avg
                eval += similarity[user_id][neighbor] * (data[neighbor]['ratings'][item_id] + data[neighbor]['sum']/data[neighbor]['num_ratings'] - baseline)
        if sim_sum != 0:
            eval /= sim_sum
        if item_id in item_rating:
            baseline = data[user_id]['sum']/data[user_id]['num_ratings'] +  item_rating[item_id]['sum']/item_rating[item_id]['num'] - all_avg
        else:
            baseline = data[user_id]['sum']/data[user_id]['num_ratings']
        eval += baseline
        if eval > 100:
            eval = 100
        test_output[user_id][item_id] = int(eval) 
            

100%|██████████| 19835/19835 [02:04<00:00, 159.60it/s]


In [ ]:
result_path = 'result_baseline.txt'
count = 0
with open(result_path, 'w') as f:
    for userid, item_list in test_output.items():
        f.write(f"{userid}|6\n")
        for item, rating in item_list.items():
            f.write(f"{item}  {rating}\n")

In [12]:
'''
w : (N,1)
true_rating: (len(items))
'''
initial_lr = 1e-3
epsilon = 100
beta1 = 0.9
beta2 = 0.999
eps = 1e-8
t = 0

lr = 1e-6
epsilon = 100
weight = {}
userid = 37
N = 50
N_neighbor = find_top_n_keys(similarity[userid], N)
w = np.array([similarity[userid][k] for k in N_neighbor])

m = np.zeros_like(w)
v = np.zeros_like(w)

rating_matrix = np.zeros((N, len(data[userid]['ratings'])))
true_rating = np.array(list(data[userid]['ratings'].values()))
user_bias = np.zeros(len(data[userid]['ratings']))

grad = np.zeros(N)
for i,itemid in enumerate(data[userid]['ratings'].keys()):
    user_bias[i] = bias[userid] + item_bias[itemid] + all_avg
    for j,neighbor in enumerate(N_neighbor):
        if itemid in data[neighbor]['ratings']: 
            rating_matrix[j][i] = data[neighbor]['ratings'][itemid]  - (bias[neighbor] + item_bias[itemid] + all_avg)
        else:
            rating_matrix[j][i] = 0

while True:
    t += 1
    eval = np.dot(w, rating_matrix) 
    nonzero_indices = np.nonzero(eval)
    eval[nonzero_indices] += user_bias[nonzero_indices]
    
    loss = np.sum((eval[nonzero_indices] - true_rating[nonzero_indices])**2)
    #grad = np.array([(2*(eval - true_rating) * rating_matrix[N_neighbor.index(n)]).sum() for n in N_neighbor])
    for j, neighbor in enumerate(N_neighbor):
        grad[j] = np.sum(2 * (eval[nonzero_indices] - true_rating[nonzero_indices]) * rating_matrix[j, nonzero_indices])

    # Adam update
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    w = w - initial_lr * m_hat / (np.sqrt(v_hat) + eps)
    #w = w - lr*grad
    print(grad)
    print(loss)
    if(np.linalg.norm(grad) < epsilon):
        break

[-568790.42372941 -373102.82603451  247026.1143081   259312.56390739
  457458.44186701  437626.5489056   411886.86442544  388008.43163719
  168077.1068042   192253.42291282  100838.0532001   395544.35533731
  515478.19535677  168731.73944888 -292807.40274714  522122.22541162
  238186.24993688  147224.40197371 -156388.42662613  371635.92761757
  194511.32882244   56686.99373633  408626.18690773 -459583.36803298
  356191.13236068  355808.05102885  223899.6759182   104145.83319002
  133897.54286472  244512.47620986  240852.58394512  227947.90880715
   68826.89543645  120453.95544982   31882.18361006  269500.77905087
  252145.88603313  343690.36183036  278457.81647837  242757.79724976
    3430.93514174  138238.70903201  166832.42503521  163173.44512544
  299229.78390108   33870.23472951  247820.04499949  157488.50058349
 -116248.69794985  378374.91678297]
4026548.3789463844
[-563946.10476244 -370556.59352232  239771.84543443  253693.78334515
  449047.87360309  428402.25808151  403878.68322

In [16]:
list(data.keys())[:40]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]

In [17]:
weight = {}
for userid in tqdm(list(data.keys())[:50]):
    initial_lr = 1e-3
    epsilon = 100
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-8
    t = 0

    N = 50
    N_neighbor = find_top_n_keys(similarity[userid], N)
    w = np.array([similarity[userid][k] for k in N_neighbor])

    m = np.zeros_like(w)
    v = np.zeros_like(w)

    rating_matrix = np.zeros((N, len(data[userid]['ratings'])))
    true_rating = np.array(list(data[userid]['ratings'].values()))
    user_bias = np.zeros(len(data[userid]['ratings']))

    grad = np.zeros(N)
    for i,itemid in enumerate(data[userid]['ratings'].keys()):
        user_bias[i] = bias[userid] + item_bias[itemid] + all_avg
        for j,neighbor in enumerate(N_neighbor):
            if itemid in data[neighbor]['ratings']: 
                rating_matrix[j][i] = data[neighbor]['ratings'][itemid]  - (bias[neighbor] + item_bias[itemid] + all_avg)
            else:
                rating_matrix[j][i] = 0

    while True:
        t += 1
        eval = np.dot(w, rating_matrix) 
        nonzero_indices = np.nonzero(eval)
        eval[nonzero_indices] += user_bias[nonzero_indices]
        
        loss = np.sum((eval[nonzero_indices] - true_rating[nonzero_indices])**2)
        #grad = np.array([(2*(eval - true_rating) * rating_matrix[N_neighbor.index(n)]).sum() for n in N_neighbor])
        for j, neighbor in enumerate(N_neighbor):
            grad[j] = np.sum(2 * (eval[nonzero_indices] - true_rating[nonzero_indices]) * rating_matrix[j, nonzero_indices])

        # Adam update
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        w = w - initial_lr * m_hat / (np.sqrt(v_hat) + eps)
        #w = w - lr*grad
        if(np.linalg.norm(grad) < epsilon):
            break
    
    weight[userid] = {k: v for k, v in zip(N_neighbor, w)}

100%|██████████| 50/50 [03:34<00:00,  4.29s/it]


In [ ]:
'''
w : (N,1)
true_rating: (len(items))
'''
lr = 1e-6
epsilon = 100
weight = {}
userid = 2
N = 50
N_neighbor = find_top_n_keys(similarity[userid], N)
w = np.array([similarity[userid][k] for k in N_neighbor])

rating_matrix = np.zeros((N, len(data[userid]['ratings'])))
true_rating = np.array(list(data[userid]['ratings'].values()))
user_bias = np.zeros(len(data[userid]['ratings']))

grad = np.zeros(N)
for i,itemid in enumerate(data[userid]['ratings'].keys()):
    user_bias[i] = bias[userid] + item_bias[itemid] + all_avg
    for j,neighbor in enumerate(N_neighbor):
        if itemid in data[neighbor]['ratings']: 
            rating_matrix[j][i] = data[neighbor]['ratings'][itemid]  - (bias[neighbor] + item_bias[itemid] + all_avg)
        else:
            rating_matrix[j][i] = 0

while True:
    eval = np.dot(w, rating_matrix) 
    nonzero_indices = np.nonzero(eval)
    eval[nonzero_indices] += user_bias[nonzero_indices]
    
    loss = np.sum((eval[nonzero_indices] - true_rating[nonzero_indices])**2)
    #grad = np.array([(2*(eval - true_rating) * rating_matrix[N_neighbor.index(n)]).sum() for n in N_neighbor])
    for j, neighbor in enumerate(N_neighbor):
        grad[j] = np.sum(2 * (eval[nonzero_indices] - true_rating[nonzero_indices]) * rating_matrix[j, nonzero_indices])

    w = w - lr*grad
    print(grad)
    print(loss)
    if(np.linalg.norm(grad) < epsilon):
        break


In [ ]:
#set centerlize = False
lr = 1e-6
weight = {}
userid = 1
N = 50
weight[userid] = {k:similarity[userid][k] for k in find_top_n_keys(similarity[userid], 50)}
true_rating = {id: r for (id,r) in data[userid]['ratings'].items()}
epsilon = 100

while True:
    estimate_rating = {id: 0 for id in data[userid]['ratings'].keys()}
    grad = {id: 0 for id in weight[userid].keys()}
    loss = 0

    for itemid in data[userid]['ratings'].keys():
        for neighbor in weight[userid].keys():
            if itemid in data[neighbor]['ratings']:    #user and its neighbor's common item
                            #original rating
                r_minus_b = data[neighbor]['ratings'][itemid]  - (bias[neighbor] + item_bias[itemid] + all_avg)#bias of item
                                                     
                estimate_rating[itemid] += weight[userid][neighbor] * r_minus_b
        
        if estimate_rating[itemid] == 0:
            continue

        estimate_rating[itemid] += (bias[userid] + item_bias[itemid] + all_avg)
        loss += (estimate_rating[itemid] - true_rating[itemid]) **2
    
    print(loss)
    
    for neighbor in weight[userid].keys():
        for itemid in data[userid]['ratings'].keys():
            if itemid in data[neighbor]['ratings']:
                r_minus_b = data[neighbor]['ratings'][itemid]  - (bias[neighbor] + item_bias[itemid] + all_avg)                                         
                grad[neighbor] += 2*(estimate_rating[itemid] - true_rating[itemid]) * r_minus_b
    print(grad)
    weight[userid].update({k: v - lr*grad[k] for k, v in weight[userid].items()})
    grad_vec = list(grad.values())
    grad_vec_sum = np.sum(np.abs(grad_vec))
    if (np.linalg.norm(grad_vec)) < epsilon:
        print(true_rating)
        print(estimate_rating)
        break

In [ ]:
lr = 1e-7
weight = {}
userid = 37
weight[userid] = {k:similarity[userid][k] for k in find_top_n_keys(similarity[userid], 100)}
while True:
    epsilon = 1e-5
    true_rating = {id:r+data[userid]['sum']/data[userid]['num_ratings'] for (id,r) in data[userid]['ratings'].items()}
    estimate_rating = {id: 0 for id in data[userid]['ratings'].keys()}
    grad = {id: 0 for id in weight[userid].keys()}
    loss = 0

    for itemid in data[userid]['ratings'].keys():
        for neighbor in weight[userid].keys():
            if itemid in data[neighbor]['ratings']:    #user and its neighbor's common item
                            #original rating
                r_minus_b = data[neighbor]['ratings'][itemid] + data[neighbor]['sum']/data[neighbor]['num_ratings'] - \
                                                    (bias[neighbor] + item_bias[itemid] + all_avg) #bias of item

                estimate_rating[itemid] += weight[userid][neighbor] * r_minus_b
        
        if estimate_rating[itemid] == 0:
            continue

        estimate_rating[itemid] += (bias[userid] + item_bias[itemid] + all_avg)
        loss += (estimate_rating[itemid] - true_rating[itemid]) **2
    
    print(loss)
    #print(true_rating)
    #print(estimate_rating)
    
    for neighbor in weight[userid].keys():
        for itemid in data[userid]['ratings'].keys():
            if itemid in data[neighbor]['ratings']:
                
                r_minus_b = data[neighbor]['ratings'][itemid] + data[neighbor]['sum']/data[neighbor]['num_ratings'] - \
                                                        (bias[neighbor] + item_bias[itemid] + all_avg)

                grad[neighbor] += 2*(estimate_rating[itemid] - true_rating[itemid]) * r_minus_b
                #print(neighbor,r_minus_b, grad[neighbor])
    
    weight[userid].update({k: v - lr*grad[k] for k, v in weight[userid].items()})
    grad_vec = list(grad.values())
    #np.linalg.norm(grad_vec)
    #print(grad)
    if (np.linalg.norm(grad_vec) * lr) < epsilon:
        print(grad)
        print(true_rating)
        print(estimate_rating)
        break

In [41]:
lr = 1e-6
weight = {}
for userid in tqdm(data.keys()):
    weight[userid] = {k:similarity[userid][k] for k in find_top_n_keys(similarity[userid], 10)}
    while True:
        epsilon = 0.1
        true_rating = {id:r+data[userid]['sum']/data[userid]['num_ratings'] for (id,r) in data[userid]['ratings'].items()}
        estimate_rating = {id: 0 for id in data[userid]['ratings'].keys()}
        grad = {id: 0 for id in weight[userid].keys()}
        loss = 0

        for itemid in data[userid]['ratings'].keys():
            for neighbor in weight[userid].keys():
                if itemid in data[neighbor]['ratings']:    #user and its neighbor's common item
                                #original rating
                    r_minus_b = data[neighbor]['ratings'][itemid] + data[neighbor]['sum']/data[neighbor]['num_ratings'] - \
                                                        (bias[neighbor] + item_bias[itemid] + all_avg) #bias of item

                    estimate_rating[itemid] += weight[userid][neighbor] * r_minus_b
            
            if estimate_rating[itemid] == 0:
                continue

            estimate_rating[itemid] += (bias[userid] + item_bias[itemid] + all_avg)
            loss += (estimate_rating[itemid] - true_rating[itemid]) **2
        
        #print(loss)
        
        for neighbor in weight[userid].keys():
            for itemid in data[userid]['ratings'].keys():
                if itemid in data[neighbor]['ratings']:
                    
                    r_minus_b = data[neighbor]['ratings'][itemid] + data[neighbor]['sum']/data[neighbor]['num_ratings'] - \
                                                            (bias[neighbor] + item_bias[itemid] + all_avg)

                    grad[neighbor] += 2*(estimate_rating[itemid] - true_rating[itemid]) * r_minus_b
                    ###print(neighbor,r_minus_b, grad[neighbor])
        
        weight[userid].update({k: v - lr*grad[k] for k, v in weight[userid].items()})
        grad_vec = list(grad.values())
        #np.linalg.norm(grad_vec)
        #print(grad)
        if (np.linalg.norm(grad_vec) * lr) < epsilon:
            break


  0%|          | 37/19835 [00:04<41:23,  7.97it/s] 


OverflowError: (34, 'Result too large')

In [ ]:
test_output = {}

for user_id, user_data in (tqdm(test_input.items())):
    user_id = int(user_id)
    test_output[user_id] = {}
    for item_id in user_data['items']:
        item_id = int(item_id)
        eval = 0
        for i,w in weight[userid].items():
            if item_id in data[i]['ratings']:
                r_minus_b = data[i]['ratings'][item_id] + data[i]['sum']/data[i]['num_ratings'] - \
                                                      bias[i] + item_bias[item_id] - all_avg
                eval += similarity[user_id][neighbor] * (data[neighbor]['ratings'][item_id] + data[neighbor]['sum']/data[neighbor]['num_ratings'] - baseline)
        if sim_sum != 0:
            eval /= sim_sum
        if item_id in item_rating:
            baseline = data[user_id]['sum']/data[user_id]['num_ratings'] +  item_rating[item_id]['sum']/item_rating[item_id]['num'] - all_avg
        else:
            baseline = data[user_id]['sum']/data[user_id]['num_ratings']
        eval += baseline
        if eval > 100:
            eval = 100
        test_output[user_id][item_id] = int(eval) 

In [ ]:
#the oridinary version
test_output = {}

for user_id, user_data in (tqdm(test_input.items())):
    user_id = int(user_id)
    test_output[user_id] = {}
    N_neighbor = find_top_n_keys(similarity[user_id], 500)
    for item_id in user_data['items']:
        item_id = int(item_id)
        eval = 0
        sim_sum = 0
        for neighbor in N_neighbor:
            if item_id in data[neighbor]['ratings']:
                sim_sum += similarity[user_id][neighbor]
                eval += similarity[user_id][neighbor] * (data[neighbor]['ratings'][item_id] + data[neighbor]['sum']/data[neighbor]['num_ratings'])
        if sim_sum != 0:
            eval /= sim_sum
        test_output[user_id][item_id] = int(eval)
            

print('evaluate done')

result_path = 'result_normal.txt'
count = 0
with open(result_path, 'w') as f:
    for userid, item_list in test_output.items():
        f.write(f"{userid}|6\n")
        for item, rating in item_list.items():
            f.write(f"{item}  {rating}\n")

print('write done')

100%|██████████| 19835/19835 [02:07<00:00, 155.97it/s]


evaluate done
write done


In [ ]:
with open('sim_top_10.txt', 'w') as f:
    for i, (userid1, user1_data) in enumerate(similarity.items()):
        if(i<=10):
            f.write(f"{userid1}: \n ")
            for userid2, cos_sim in user1_data.items():
                f.write(f"{userid2} {cos_sim}\n")
        else:
            break